In [ ]:
"""
The Autonomous Cooperative Consensus Orbit Determination (ACCORD) framework.
Author: Beth Probert
Email: beth.probert@strath.ac.uk

Copyright (C) 2025 Applied Space Technology Laboratory

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

"""

import json
import hashlib # TODO - might use a different algorithm? we will see if needed. Start small ffs dont get overwhelmed by the whole ledger
from typing import Optional
import numpy as np

# Global Variables for consensus
# TODO - May tune these to optimise performance
CONFIRMATION_THRESHOLD = 1
REJECTION_THRESHOLD = -1
CONFIRMATION_STEP = 0.1 

In [9]:
def load_json_data(file_name: str) -> dict:
    """
    Turns json data in a file into a Python dict.
    """
    with open(file_name, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
class Transaction():
    """
    Transaction containing information to be submitted in the Distributed Ledger.
    Consensus needs to be reached on the validity of the information received.
    """
    def __init__(self, sender_address, recipient_address, sender_private_key, timestamp, tx_data) -> None:
        self.sender_address: int = sender_address
        self.recipient_address: int = recipient_address
        self.sender_private_key: str = sender_private_key
        self.timestamp: str = timestamp
        # TODO Add cooperative OD information to TLE data json
        # tx_data will include cooperative OD data, so including the ID 
        # and trajectory info for a witnessed satellite as well.
        self.tx_data: str = tx_data
        
        self.parent_hashes: list[str] = []
        self.confirmation_score: float = 0
        self.consensus_reached: bool = False
        self.is_confirmed: bool = False
        self.is_rejected: bool = False

        self.hash = self.calculate_hash()
        
    def calculate_hash(self) -> str:
        """
        Calculates a hash string for a transaction. The hash is generated by encoding all of
        the transaction's unique information, so that ANY change to a transaction results in a new 
        hash, making tampering easier to identify.
        """
        return hashlib.sha256(str(self.sender_address).encode() + self.timestamp.encode() + 
                              self.sender_private_key.encode() + self.tx_data.encode() + 
                              str(self.recipient_address).encode()
                             ).hexdigest()

In [ ]:
class DAG():
    """
    A class representing the Directed Acyclic Graph (DAG) Distributed Ledger Technology.
    When a transaction is received, it is added to the DAG. The number of parents
    for each transaction is decided using a tip selection algorithm.
    """
    
    def __init__(self) -> None:
        # TODO - need to change from a chain with appends into a DAG with parents
        # TODO - need a way to check that the DAG has not been tampered with
        # TODO - could be a numpy array instead - maybe it should be for 2d
        self.chain: np.typing.ArrayLike = [self.create_genesis_tx()]
    
    def create_genesis_tx(self) -> Transaction:
        """
        Creates the genesis transaction to initialise the DAG and provide a parent
        for the first real transaction.
        """
        return Transaction(0, 0, "1234", "24/06/2025", "Genesis Transaction")
        
    def check_thresholds(self, transaction: Transaction) -> None:
        """
        Check if the transaction confirmation or rejection thresholds have been crossed. 
        This will affect weighting. is_confirmed = strong weighting, else weak weighting
        """
        if REJECTION_THRESHOLD <= transaction.confirmation_score <= CONFIRMATION_THRESHOLD:
            transaction.confirmation_score +=  CONFIRMATION_STEP
        else:
            transaction.confirmation_score -= CONFIRMATION_STEP
        
        if transaction.confirmation_score >= CONFIRMATION_THRESHOLD:
            transaction.is_confirmed = True
        elif transaction.confirmation_score <= REJECTION_THRESHOLD:
            transaction.is_rejected = True
    
    def get_latest_tx(self) -> Transaction:
        """
        Retrieves the most recently created transaction stored on the chain.
        TODO - may not be needed? For parent selection
        """
        return self.chain[len(self.chain) - 1]
    
    def get_second_latest_tx(self) -> Optional[Transaction]:
        """
        Retrieves the second-most recently created transaction stored on the chain.
        TODO - may not be needed? For parent selection
        """
        if len(self.chain) >= 2:
            return self.chain[len(self.chain) - 2]
        else:
            return None
    
    def add_tx(self, transaction: Transaction) -> None:
        """
        Add a transaction to the DAG
        """
        # TODO Just append for now, sort out tip selection later and number of parents
        # TODO - tx or blocks?? start with tx for now
        # TODO - fixed number of parents: 2
        parent1 = self.get_latest_tx()
        parent2 = self.get_second_latest_tx()
        
        # There is guaranteed to be one parent - the genesis transaction in the chain.
        # The first transaction will only have one parent. All others will have >= 2.
        transaction.parent_hashes.append(parent1)
        if parent2 is not None:
            transaction.parent_hashes.append(parent1)
        self.chain.append(transaction)
        self.check_thresholds(transaction)
        # THRESHOLDS affect tip selection for parents - TODO

In [ ]:
class ConsensusMechanism():
    """
    The Proof of Inter-Satellite Evaluation (PoISE) consensus mechanism.
        
    TODO - need to check: if physically possible, how many times its been seen (maybe new param?)
    # Node needs an ID, trust factor number
    """
    # TODO - proof of Location XYO paper, and bidirectional heuristics (need to check for invalid, or valid but incorrect/inaccurate)(4.3, proof of proximity)
    # TODO - format for data? class for verification/consensus? Need some calcs
    # TODO TODAY - SLAM DRONE PAPER, and consensus on position, doppler shift?? what data so I have to choose from?
    # DAGmap - map consensus could be something to build upon? does it have a ground truth?
    # PowerGraph - consensus for trust level, calculates validity of transacion from probability level. I guess I need to calculate validity from some maths? possible - if yes, hen how accurate/likely
    # probability distruibution for observations? Algorithm one in PowerGraph paper
    
    def __init__(self):
        self.consensus_threshold : float = 1.0 # TODO - tune
        
    def proof_of_something(self, dag: DAG) -> bool:
        """
        TODO - needs a name. Not sure how its gonna work yet. 
        Returns a bool of it consensus has been reached
        TODO - could get transaction from DAG, or from another arg?
        
        """
        # 1)  Get the data 
        od_data = load_json_data("od_data.json")
        
        # 2) TODO Check if data is valid, if not - ignore. If yes, add to DAG
        
        # 3) TODO Check we have enough data to be bft (3f + 1)
        # if not, consensus cannot be reached. 
        if len(dag.chain) < 4:
            return False
        
        # 4) TODO Check if satellite has been witnessed before
        #4a if yes, does this data agree with other data/ is it correct? Assign correctness score -> affects transaction
        
        # 5) TODO is sensor data accurate (done regardless of previous witnessing). Assign accuracy score -> affects transaction and node reputation
        
        # 6) TODO calculate consensus score - node reputation, accuracy and correctness all factor
        
        # 7) TODO if consensus score above threshold, consensus reached. Else not.

# Thoughts
Options are:
1. Satellites send their own positional and trajectory info, along with IDs of any satellites they witness
    - No external measurements needed, just basic witnessing
    - Satellite A reports: ' I saw satellite B and here is my positional data'. Satellite B reports 'I saw satellite A and here is my positional data'. Can verify that these are correct within a certain error tolerance. Is it mathematically possible they saw each other?
2. Satellites send the position and and velocity info of a satellite they witness
    - Requires accurate measurement of another satellite's position and velocity from another satellite. Not sure every satellite can do this. 
    - Stronger witnessing, but larger data set to send. 

- Need to consider doppler shift, TLE data, how can satellites actually witness each other? Through ISLs and broadcasting?
- Could it be verified against existing ground data to begin with? Then moved away with ground as a back up for redundancy?
- What calculations would need to be performed? 
- Incentivised to provide honest information with reputation score, then consensus reached later. 
- Use cases: Invalid info, valid but incorrect info: https://docs.xyo.network/XYO-White-Paper.pdf (Sections 4.2 and 4.3)
- Do I want a constant chain, or a chain that gets queried and then develops? An API? How will users interact? (Maybe a question for later)
- XYO - do I need a signature? What if the node is compromised? I guess maybe it tells me it is? 


# References 
* https://github.com/samuelxu999/EconLedger/blob/main/src/consensus/ENF_consensus.py
* https://github.com/helium/oracles/blob/main/iot_verifier/src/poc.rs
* https://www.swirlds.com/downloads/SWIRLDS-TR-2016-01.pdf
* https://docs.helium.com/iot/proof-of-coverage-roadmap
* https://www.researchgate.net/publication/379723426_Decentralized_And_Neutral_Consensus_Mechanisms_in_Space_Conjunctions_Assessment_and_Mitigation_Space_DAO_STM
* https://www.sciencedirect.com/science/article/pii/S0094576524003308
* https://www.mdpi.com/2504-446X/6/2/34
* https://www.mdpi.com/1996-1073/12/18/3570
* https://www.linkedin.com/posts/spacecoin-official_proof-of-location-activity-7336127810270375937-TX8U/
* https://spacecoin.org/ 
* decentralized physical infrastructure network: https://ieeexplore.ieee.org/document/10737386
* https://docs.hivemapper.com/, https://geodnet.com/
* https://docs.natix.network/whitepaper/natix-network-ecosystem/data-validation
* location verification first emerged in 2016: https://docs.xyo.network/XYO-White-Paper.pdf (Sections 2, 4.2 and 4.3)

![Consensus Flowchart](images/consensus_flowchart.png)